# Safety Evaluators with the Azure AI Evaluation SDK
The following sample shows the basic way to evaluate a Generative AI application in your development environment with the Azure AI evaluation SDK.

> ✨ ***Note*** <br>
> Please check the reference document before you get started - https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/evaluate-sdk

## 🔨 Current Support and Limitations (as of 2025-01-14) 
- Check the region support for the Azure AI Evaluation SDK. https://learn.microsoft.com/en-us/azure/ai-studio/concepts/evaluation-metrics-built-in?tabs=warning#region-support

### Region support for evaluations
| Region              | Hate and Unfairness, Sexual, Violent, Self-Harm, XPIA, ECI (Text) | Groundedness (Text) | Protected Material (Text) | Hate and Unfairness, Sexual, Violent, Self-Harm, Protected Material (Image) |
|---------------------|------------------------------------------------------------------|---------------------|----------------------------|----------------------------------------------------------------------------|
| North Central US    | no                                                               | no                  | no                         | yes                                                                        |
| East US 2           | yes                                                              | yes                 | yes                        | yes                                                                        |
| Sweden Central      | yes                                                              | yes                 | yes                        | yes                                                                        |
| US North Central    | yes                                                              | no                  | yes                        | yes                                                                        |
| France Central      | yes                                                              | yes                 | yes                        | yes                                                                        |
| Switzerland West    | yes                                                              | no                  | no                         | yes                                                                        |

### Region support for adversarial simulation
| Region            | Adversarial Simulation (Text) | Adversarial Simulation (Image) |
|-------------------|-------------------------------|---------------------------------|
| UK South          | yes                           | no                              |
| East US 2         | yes                           | yes                             |
| Sweden Central    | yes                           | yes                             |
| US North Central  | yes                           | yes                             |
| France Central    | yes                           | no                              |


## ✔️ Pricing and billing
- Effective 1/14/2025, Azure AI Safety Evaluations will no longer be free in public preview. It will be billed based on consumption as following:

| Service Name              | Safety Evaluations       | Price Per 1K Tokens (USD) |
|---------------------------|--------------------------|---------------------------|
| Azure Machine Learning    | Input pricing for 3P     | $0.02                     |
| Azure Machine Learning    | Output pricing for 3P    | $0.06                     |
| Azure Machine Learning    | Input pricing for 1P     | $0.012                    |
| Azure Machine Learning    | Output pricing for 1P    | $0.012                    |


In [ ]:
import pandas as pd
import os
import json

from pprint import pprint
from azure.ai.evaluation import evaluate
from azure.ai.evaluation import RelevanceEvaluator
from azure.ai.evaluation import GroundednessEvaluator, GroundednessProEvaluator
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    Evaluation,
    Dataset,
    EvaluatorConfiguration,
    ConnectionType,
    EvaluationSchedule,
    RecurrenceTrigger,
    ApplicationInsightsConfiguration,
)
import pathlib

from azure.ai.evaluation import evaluate
from azure.ai.evaluation import (
    ContentSafetyEvaluator,
    IndirectAttackEvaluator,
)
from azure.ai.evaluation.simulator import (
    AdversarialSimulator,
    AdversarialScenario,
    AdversarialScenarioJailbreak,
    IndirectAttackSimulator,
)
from azure.ai.evaluation.simulator._adversarial_scenario import (
    _UnstableAdversarialScenario,
)
from openai import AzureOpenAI
from typing import List, Dict, Optional, Any


load_dotenv(override=True)

In [ ]:
# Initialize Azure AI project and Azure OpenAI conncetion with your environment variables
azure_ai_project_conn_str = os.environ.get("AZURE_AI_PROJECT_CONN_STR")
subscription_id = azure_ai_project_conn_str.split(";")[1]
resource_group_name = azure_ai_project_conn_str.split(";")[2]
project_name = azure_ai_project_conn_str.split(";")[3]

azure_ai_project = {
    "subscription_id": subscription_id,
    "resource_group_name": resource_group_name,
    "project_name": project_name,
}

azure_openai_deployment = os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME")
azure_openai_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
azure_openai_key = (os.environ.get("AZURE_OPENAI_API_KEY"),)
azure_openai_api_version = os.environ.get("AZURE_OPENAI_API_VERSION")

credential = DefaultAzureCredential()

In [ ]:
query_response = dict(
    query="Which tent is the most waterproof?",
    context="The Alpine Explorer Tent is the most water-proof of all tents available.",
    response="The Alpine Explorer Tent is the most waterproof.",
)

conversation_str = """{"messages": [ { "content": "Which tent is the most waterproof?", "role": "user" }, { "content": "The Alpine Explorer Tent is the most waterproof", "role": "assistant", "context": "From the our product list the alpine explorer tent is the most waterproof. The Adventure Dining Table has higher weight." }, { "content": "How much does it cost?", "role": "user" }, { "content": "$120.", "role": "assistant", "context": "The Alpine Explorer Tent is $120."} ] }"""
conversation = json.loads(conversation_str)

## 🧪 AI-assisted ContentSafety evaluator
- Combines all the safety evaluators - ViolenceEvaluator, SexualEvaluator, SelfHarmEvaluator and HateUnfairnessEvaluator for a single output of combined metrics for query and response pairs
- Evaluating direct attack is a comparative measurement using the content safety evaluators as a control.

[Note] If you encounter the following error, please `az login` and try again. You can also check the Azure AI Evaluation SDK documentation for more information.

> EvaluationException: (UserError) Failed to connect to your Azure AI project. Please check if the project scope is configured correctly, and make sure you have the necessary access permissions. Status code: 401

In [ ]:
# !az login --scope https://graph.microsoft.com//.default

In [ ]:
contentsafety_eval = ContentSafetyEvaluator(
    credential=credential, azure_ai_project=azure_ai_project
)

uery_response = dict(
    query="Which tent is the most waterproof?",  # optional
    context="The Alpine Explorer Tent is the most water-proof of all tents available.",
    response="The Alpine Explorer Tent is the most waterproof.",
)

contentsafety_score = contentsafety_eval(**query_response)
print(contentsafety_score)

contentsafety_conv_score = contentsafety_eval(conversation=conversation)
print(contentsafety_conv_score)

## 🧪 AI-assisted ContentSafety evaluator (Image)
- In this scenario the Adversarial Simulator provides harmful text prompts, which can be used in your GenAI application or LLMs to generate image(s) and use these Text + Image prompts for evaluation purposes. Simulator expects an assistant message with image(s) to be added in the conversation within the callback function. See example below:

In [ ]:
def call_gen_ai_application_or_llm(user_prompt, system_prompt) -> str:
    client = AzureOpenAI(
        api_version=azure_openai_api_version,
        azure_endpoint=azure_openai_endpoint,
        api_key=azure_openai_key,
    )

    # Call the model
    messages = []
    messages.append(
        {
            "role": "system",
            "content": system_prompt,
        }
    )
    messages.append(
        {
            "role": "user",
            "content": user_prompt,
        }
    )

    completion = client.chat.completions.create(
        model=azure_openai_deployment,
        messages=messages,
    )

    response = completion.to_dict()["choices"][0]["message"]
    if type(response) == dict:
        content = response["content"]
    return content


async def call_llm_image_generation(query: str) -> str:
    dalle_client = AzureOpenAI(
        api_version=azure_openai_api_version,
        azure_endpoint=azure_openai_endpoint,
        api_key=azure_openai_key,
    )

    result = dalle_client.images.generate(
        model="dall-e-3", prompt=query, n=1, size="1024x1024"
    )

    print(result)

    return result.data[0].url


async def callback(
    messages: List[Dict],
    stream: bool = False,
    session_state: Any = None,
    context: Optional[Dict[str, Any]] = None,
) -> dict:
    image_understanding_prompt = messages["messages"][0]["content"]
    content = await call_gen_ai_application_or_llm(
        image_understanding_prompt, "You are an AI assistant who can describe images."
    )
    formatted_response = {"content": content, "role": "assistant"}
    messages["messages"].append(formatted_response)
    return {
        "messages": messages["messages"],
        "stream": stream,
        "session_state": session_state,
        "context": context,
    }

In [ ]:
adversarial_image_gen_simulator = AdversarialSimulator(
    azure_ai_project=azure_ai_project, credential=credential
)

simulator_output = await adversarial_image_gen_simulator(
    scenario=_UnstableAdversarialScenario.ADVERSARIAL_IMAGE_GEN, target=callback
)


pprint(simulator_output)

In [ ]:
file_name = "data/eval_sim_test.jsonl"
with open(file_name, "w") as file:
    file.writelines(
        [
            json.dumps({"conversation": {"messages": conversation["messages"]}}) + "\n"
            for conversation in simulator_output
        ]
    )

# run the evaluation on the simulation results
content_safety_eval = ContentSafetyEvaluator(credential, azure_ai_project)

eval_output = evaluate(
    data=file_name,
    evaluation_name="sim_image_gen_content_safety_eval",
    azure_ai_project=azure_ai_project,
    evaluators={"content_safety": content_safety_eval},
)

In [ ]:
image_understanding_simulator = AdversarialSimulator(
    azure_ai_project=azure_ai_project, credential=credential
)

simulator_output = await image_understanding_simulator(
    scenario=_UnstableAdversarialScenario.ADVERSARIAL_IMAGE_MULTIMODAL, target=callback
)


pprint(simulator_output)

In [ ]:
file_name = "data/eval_sim_test2.jsonl"
with open(file_name, "w") as file:
    file.writelines(
        [
            json.dumps({"conversation": {"messages": conversation["messages"]}}) + "\n"
            for conversation in simulator_output
        ]
    )

# run the evaluation on the simulation results
content_safety_eval = ContentSafetyEvaluator(credential, azure_ai_project)

eval_output = evaluate(
    data=file_name,
    evaluation_name="sim_multimodel_image_understanding_content_safety_eval",
    azure_ai_project=azure_ai_project,
    evaluators={"content_safety": content_safety_eval},
)

In [ ]:
async def xpia_callback(
    messages: List[Dict],
    stream: bool = False,
    session_state: Optional[str] = None,
    context: Optional[Dict] = None,
) -> dict:
    messages_list = messages["messages"]
    # get last message
    latest_message = messages_list[-1]
    query = latest_message["content"]
    context = None
    if "file_content" in messages["template_parameters"]:
        query += messages["template_parameters"]["file_content"]
    # the next few lines explain how to use the AsyncAzureOpenAI's chat.completions
    # to respond to the simulator. You should replace it with a call to your model/endpoint/application
    # make sure you pass the `query` and format the response as we have shown below

    # Get a client handle for the model
    client = AzureOpenAI(
        azure_endpoint=azure_openai_endpoint,
        api_version=azure_openai_api_version,
        api_key=azure_openai_key,
    )
    try:
        response_from_oai_chat_completions = client.chat.completions.create(
            messages=[{"content": query, "role": "user"}],
            model=azure_openai_deployment,
            max_tokens=300,
        )
        print(response_from_oai_chat_completions)
    except Exception as e:
        print(f"Error: {e} with content length {len(query)}")
        # to continue the conversation, return the messages, else you can fail the adversarial with an exception
        message = {
            "content": "Something went wrong. Check the exception e for more details.",
            "role": "assistant",
            "context": None,
        }
        messages["messages"].append(message)
        return {
            "messages": messages["messages"],
            "stream": stream,
            "session_state": session_state,
        }
    response_result = response_from_oai_chat_completions.choices[0].message.content
    formatted_response = {
        "content": response_result,
        "role": "assistant",
        "context": {},
    }
    messages["messages"].append(formatted_response)
    return {
        "messages": messages["messages"],
        "stream": stream,
        "session_state": session_state,
        "context": context,
    }

In [ ]:
indirect_attack_simulator = IndirectAttackSimulator(
    azure_ai_project=azure_ai_project, credential=DefaultAzureCredential()
)

unfiltered_indirect_attack_outputs = await indirect_attack_simulator(
    target=xpia_callback,
    scenario=AdversarialScenarioJailbreak.ADVERSARIAL_INDIRECT_JAILBREAK,
    max_simulation_results=10,
    max_conversation_turns=3,
)

In [ ]:
pprint(unfiltered_indirect_attack_outputs)

In [ ]:
# Results are truncated for brevity.
truncation_limit = 50
for output in unfiltered_indirect_attack_outputs:
    for turn in output["messages"]:
        content = turn["content"]
        if isinstance(content, dict):  # user response from callback is dict
            print(f"{turn['role']} : {content['content'][0:truncation_limit]}")
        elif isinstance(content, tuple):  # assistant response from callback is tuple
            print(f"{turn['role']} : {content[0:truncation_limit]}")

In [ ]:
from pathlib import Path

print(unfiltered_indirect_attack_outputs)
print(unfiltered_indirect_attack_outputs.to_eval_qr_json_lines())
output = unfiltered_indirect_attack_outputs.to_eval_qr_json_lines()
xpia_file_path = "unfiltered_indirect_attack_outputs.jsonl"

# Write the output to the file
with Path.open(Path(xpia_file_path), "w") as file:
    file.write(output)